**Membros** <br>
André de Souza Gonçalves <br>
Victor Teodoro Goraieb <br>
Vitor Anastácio 


# Pacote de dados


*   Abra o link do pacote de dados: https://tinyurl.com/bd10-auxilio
*   Selecione a opção "Adicionar Atalho ao Drive"


In [ ]:
!pip install mrjob

     |████████████████████████████████| 439 kB 5.1 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/auxilio/"

 auxilio_0.1pct.csv.gz
 auxilio_10pct.csv.gz
 auxilio_1pct.csv.gz
 auxilio.csv
 auxilio.csv.gz
'Dicionário de Dados - Auxílio Emergencial - Portal da transparência.pdf'
 estimativa_dou_2021.csv


In [ ]:
!zcat "/content/drive/My Drive/auxilio/auxilio.csv.gz" | head

"MÊS DISPONIBILIZAÇÃO";"UF";"CÓDIGO MUNICÍPIO IBGE";"NOME MUNICÍPIO";"NIS BENEFICIÁRIO";"CPF BENEFICIÁRIO";"NOME BENEFICIÁRIO";"NIS RESPONSÁVEL";"CPF RESPONSÁVEL";"NOME RESPONSÁVEL";"ENQUADRAMENTO";"PARCELA";"OBSERVAÇÃO";"VALOR BENEFÍCIO"
"202011";"";"";"";"00000000000";"***.643.870-**";"";"-2";"";"Não se aplica";"EXTRA CADUN";"7ª";"Não há";"300,00"
"202011";"";"";"";"00000000000";"***.108.816-**";"";"-2";"";"Não se aplica";"CADUN NAO BOLSA";"6ª";"Não há";"300,00"
"202011";"";"";"";"00000000000";"***.095.215-**";"";"-2";"";"Não se aplica";"CADUN NAO BOLSA";"6ª";"Não há";"300,00"
"202011";"";"";"";"00000000000";"***.108.816-**";"";"-2";"";"Não se aplica";"CADUN NAO BOLSA";"7ª";"Não há";"300,00"
"202011";"";"";"";"00000000000";"***.885.226-**";"";"-2";"";"Não se aplica";"CADUN NAO BOLSA";"5ª";"Não há";"600,00"
"202011";"";"";"";"00000000000";"***.885.226-**";"";"-2";"";"Não se aplica";"CADUN NAO BOLSA";"3ª";"Não há";"600,00"
"202011";"";"";"";"00000000000";"***.885.226-**";"";"-2";"";"Nã

In [ ]:
%%file auxilio.py
from mrjob.job import MRJob
from mrjob.protocol import TextProtocol
import re
import logging

class Auxilio(MRJob):

  OUTPUT_PROTOCOL = TextProtocol

  def configure_args(self):
    super(Auxilio, self).configure_args()
    self.add_passthru_arg('--popFile', default="/content/drive/My Drive/auxilio/estimativa_dou_2021.csv")

  def mapper(self, _, line):
    
    try :
      linha = line.split(';') # Separando as colunas de uma linha do csv
      cod_municipio = linha[2].replace('"', '') # Salvando o codigo do municipio e tratando o casos das aspas
      valor_beneficio = float(linha[13].replace(',', '.').replace('"', '')) # Convertendo o valor do beneficio para float
      yield cod_municipio, valor_beneficio # Mapeando codigo do municipio e valor recebido

    except :
      logging.warning("linha descartada: "+line)
      pass

  def reducer_init(self):
    logging.warning("criando dicionário de municipios")
    popFile = open(self.options.popFile, "r")
    self.popDict = {}
    for line in popFile :
      try :
        uf,coduf,codmun,nome,pop = line.split(',')
        self.popDict[coduf+codmun] = [nome, int(pop)]
      except:
        pass      
    popFile.close()

  def reducer(self, key, values):
    try :
      # Calculando a soma dos valores do beneficio e dividindo pela populacao para cada cidade
      yield key, str(sum(values)/self.popDict[key][1])
    except :
      pass

if __name__ == '__main__':
    logging.basicConfig(filename="log.txt")
    Auxilio.run()

Overwriting auxilio.py


In [ ]:
!python auxilio.py "/content/drive/My Drive/auxilio/auxilio_0.1pct.csv.gz" > auxilio_01pct.txt 

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/auxilio.root.20211031.184007.154771
Running step 1 of 1...
job output is in /tmp/auxilio.root.20211031.184007.154771/output
Streaming final output from /tmp/auxilio.root.20211031.184007.154771/output...
Removing temp directory /tmp/auxilio.root.20211031.184007.154771...


In [ ]:
!cat auxilio_01pct.txt

3201407	0.14846137633048792
3201506	0.1364547041831948
3201605	0.11722100447917659
3201704	0.18623418949328782
3201803	0.141643059490085
3201902	0.19953106682297772
3202009	0.1324893272486383
3202108	0.1473975734130473
3202207	0.09383797309978105
3202256	0.12462635088526099
3202306	0.168207318628076
3202405	0.17996326962584822
3202454	0.19056871683730664
3202504	0.1417211243209196
3202553	0.16987542468856173
3202603	0.10651139671944898
3202652	0.16932416617905208
3202702	0.18237734229723931
3202801	0.1982721629430443
3202900	0.14377456148758747
3203007	0.18506305877553794
3203056	0.12358732470163665
3203106	0.16301880674448768
3203130	0.13234768093478003
3203163	0.15074640534847514
3203205	0.12320658674306695
3203304	0.16833833769884368
3203320	0.1744822843169719
3203346	0.14001516830990024
3203353	0.13749904514551983
3203403	0.14321868169791788
3203502	0.1559565263268967
3203601	0.16569129480614483
3203700	0.15085002328830927
3203809	0.11536982438148956
3203908	0.14088392346948828
320

In [ ]:
!cat log.txt

INFO:mrjob.conf:No configs found; falling back on auto-configuration
INFO:mrjob.runner:Creating temp directory /tmp/auxilio.root.20211031.182238.645883
INFO:mrjob.sim:Running step 1 of 1...
INFO:mrjob.runner:job output is in /tmp/auxilio.root.20211031.182238.645883/output
INFO:mrjob.runner:Streaming final output from /tmp/auxilio.root.20211031.182238.645883/output...
INFO:mrjob.runner:Removing temp directory /tmp/auxilio.root.20211031.182238.645883...
INFO:mrjob.conf:No configs found; falling back on auto-configuration
INFO:mrjob.runner:Creating temp directory /tmp/auxilio.root.20211031.182307.873465
INFO:mrjob.sim:Running step 1 of 1...
INFO:mrjob.runner:job output is in /tmp/auxilio.root.20211031.182307.873465/output
INFO:mrjob.runner:Streaming final output from /tmp/auxilio.root.20211031.182307.873465/output...
INFO:mrjob.runner:Removing temp directory /tmp/auxilio.root.20211031.182307.873465...
INFO:mrjob.conf:No configs found; falling back on auto-configuration
INFO:mrjob.runner:C

In [ ]:
!rm log.txt

In [ ]:
!python auxilio.py "/content/drive/My Drive/auxilio/auxilio.csv.gz" > resultado.txt 

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/auxilio.root.20211031.184054.337480
Running step 1 of 1...
job output is in /tmp/auxilio.root.20211031.184054.337480/output
Streaming final output from /tmp/auxilio.root.20211031.184054.337480/output...
Removing temp directory /tmp/auxilio.root.20211031.184054.337480...


In [ ]:
!cat resultado.txt 

3201407	142.9592949731268
3201506	141.10384364716012
3201605	124.80437752152228
3201704	153.50345309226353
3201803	193.82932011331445
3201902	179.35190504103164
3202009	175.04843220962755
3202108	157.31730262001054
3202207	126.9448590196166
3202256	132.428221046984
3202306	137.70958179268138
3202405	169.90434539002678
3202454	181.29758612958673
3202504	124.38225336587671
3202553	186.6268403171008
3202603	136.4019740112192
3202652	178.37134289057929
3202702	141.7731368902875
3202801	187.2367480046915
3202900	163.35301447330585
3203007	170.8969643403474
3203056	145.11883883630378
3203106	134.67120622568095
3203130	124.59890306426612
3203163	158.31001007418263
3203205	127.25896915245751
3203304	149.1912093528397
3203320	178.9953641203291
3203346	158.9035062131731
3203353	140.076541135131
3203403	141.68380689443615
3203502	151.11913052653793
3203601	145.4908558888076
3203700	173.63384955752213
3203809	123.38398923214973
3203908	143.97339953892535
3204005	120.47191155126782
3204054	121.2935

In [ ]:
from google.colab import files
files.download('resultado.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
drive.flush_and_unmount()